# Using tip trackers

PyLabRobot has tip trackers for both the pipetting channels and the spots in the tip rack. The tip tracker is a simple class that keeps track of the current tip, and the previous operations that have been performed on an object. This enables features like {meth}`~pylabrobot.liquid_handling.LiquidHandler.return_tips` and automated tip type detection.

In [1]:
from pylabrobot.liquid_handling import LiquidHandler, SerializingSavingBackend
from pylabrobot.liquid_handling.resources.ml_star import HTF_L, TIP_CAR_480_A00
from pylabrobot.liquid_handling.resources.hamilton import STARLetDeck

lh = LiquidHandler(backend=SerializingSavingBackend(num_channels=8), deck=STARLetDeck())
lh.setup()

In [2]:
tip_carrier = TIP_CAR_480_A00(name="tip carrier") # initialize a tip carrier

## Using the tip tracker

### Initializing tip racks

Whether or not tip tracking is turned on, spots on a tip rack initialize with a tip tracker that defaults to having a tip. The tip tracker only comes into play with performing operations.

In [3]:
tip_carrier[0] = tip_rack = HTF_L(name="tip rack")

In [4]:
tip_rack.get_item("A1").tracker.has_tip

True

To initialize a tip rack without tips, pass `with_tips=False`:

In [5]:
tip_carrier[1] = empty_tip_rack = HTF_L(name="empty tip rack", with_tips=False)

In [6]:
empty_tip_rack.get_item("A1").tracker.has_tip

False

To "empty" a tip rack after initialization, use the {meth}`~pylabrobot.liquid_handling.resources.abstract.TipRack.empty()` method. To "fill" a tip rack after initialization, use the {meth}`~pylabrobot.liquid_handling.resources.abstract.TipRack.fill()` method.

In [7]:
empty_tip_rack.fill()
empty_tip_rack.get_item("A1").tracker.has_tip

True

In [8]:
empty_tip_rack.empty()
empty_tip_rack.get_item("A1").tracker.has_tip

False

In [9]:
lh.deck.assign_child_resource(tip_carrier, rails=3)

### Inspecting operation history

In [10]:
lh.pick_up_tips(tip_rack[0])

In [11]:
tip_rack.get_item("A1").tracker.has_tip

False

In [12]:
tip_rack.get_item("A1").tracker.ops

[Pickup(tip=TipSpot(name=tip rack_tipspot_0_0, location=(011.700, 072.800, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=(000.000, 000.000, 000.000))]

In [13]:
lh.drop_tips(tip_rack[0])

In [14]:
tip_rack.get_item("A1").tracker.has_tip

True

In [15]:
tip_rack.get_item("A1").tracker.ops

[Pickup(tip=TipSpot(name=tip rack_tipspot_0_0, location=(011.700, 072.800, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=(000.000, 000.000, 000.000)),
 Drop(tip=TipSpot(name=tip rack_tipspot_0_0, location=(011.700, 072.800, -83.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=(000.000, 000.000, 000.000))]

### Errors

The tip tracker is most useful for catching hardware errors before they happen. With tip tracking turned on, the following errors can be raised:

In [16]:
from pylabrobot.liquid_handling.errors import (
  ChannelHasTipError,
  ChannelHasNoTipError,
  TipSpotHasTipError,
  TipSpotHasNoTipError,
)

#### `TipSpotHasNoTipError`

This error is raised when the tip tracker is trying to access a spot that has no tip.

In [17]:
lh.pick_up_tips(tip_rack[0])
lh.drop_tips(empty_tip_rack[0])

try:
  lh.pick_up_tips(tip_rack[0])
except TipSpotHasNoTipError as e:
  print("As expected:", e)

As expected: Tip spot tip rack_tipspot_0_0 has no tip.


#### `TipSpotHasTipError`

This error is raised when the tip tracker is trying to access a spot that has a tip.

In [18]:
lh.pick_up_tips(tip_rack[1])

try:
  lh.drop_tips(empty_tip_rack[0])
except TipSpotHasTipError as e:
  print("As expected:", e)

As expected: Tip spot empty tip rack_tipspot_0_0 already has a tip.


In [19]:
lh.drop_tips(empty_tip_rack[1])

#### `ChannelHasNoTipError`

This error is raised when the tip tracker is trying to use a channel that has no tip.

In [20]:
try:
  lh.drop_tips(empty_tip_rack[2])
except ChannelHasNoTipError as e:
  print("As expected:", e)

As expected: Channel has no tip.


#### `ChannelHasTipError`

This error is raised when the tip tracker is trying to use a channel that has a tip.

In [21]:
lh.pick_up_tips(tip_rack[2])

try:
  lh.pick_up_tips(tip_rack[3])
except ChannelHasTipError as e:
  print("As expected:", e)

As expected: Channel already has tip.


## Disabling the tip tracker

The tip tracker can be disabled in three different ways, depending on the desired behavior.

### Using a context manager

The `liquid_handling` package has a {meth}`pylabrobot.liquid_handling.no_tip_tracking` context manager that can be used to disable the tip tracker for a set of operations.

In [22]:
from pylabrobot.liquid_handling import no_tip_tracking

with no_tip_tracking():
  lh.pick_up_tips(tip_rack[4])
  lh.pick_up_tips(tip_rack[4]) # no error

### For a single object

The tip tracker can be disabled for a single object by calling {meth}`pylabrobot.liquid_handling.tip_tracker.TipTracker.disable()` on the tracker object.

In [23]:
tip_rack.get_item(5).tracker.disable()

lh.pick_up_tips(tip_rack[5])
lh.pick_up_tips(tip_rack[5]) # no error

tip_rack.get_item(5).tracker.enable()

## Globally

The tip tracker can be disabled globally by using {meth}`pylabrobot.liquid_handling.set_tip_tracking`.

In [24]:
from pylabrobot.liquid_handling import set_tip_tracking

set_tip_tracking(enabled=False)

lh.pick_up_tips(tip_rack[6])
lh.pick_up_tips(tip_rack[6]) # no error

set_tip_tracking(enabled=True)